In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report

# Chargement des données avec TensorFlow (MNIST)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Mise en forme des données pour le DataFrame
df_train = pd.DataFrame({'image': x_train.tolist(), 'label': y_train})
df_test = pd.DataFrame({'image': x_test.tolist(), 'label': y_test})

# Prétraitement des données avec pandas (redimensionnement des images)
df_train['resized_image'] = df_train['image'].apply(lambda x: np.expand_dims(x, axis=-1))
df_test['resized_image'] = df_test['image'].apply(lambda x: np.expand_dims(x, axis=-1))

# Convertir les labels en one-hot encoding avec to_categorical
y_train = to_categorical(df_train['label'], num_classes=10)
y_test = to_categorical(df_test['label'], num_classes=10)

# Séparation des données avec scikit-learn
X_train, X_val, y_train, y_val = train_test_split(df_train['resized_image'], y_train, test_size=0.2, random_state=42)

# Construction du modèle avec Keras (CNN)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compilation du modèle
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
history = model.fit(np.array(X_train.tolist()), y_train, epochs=10, batch_size=64, validation_data=(np.array(X_val.tolist()), y_val), verbose=0)
#model.fit(np.array(X_train.tolist()), y_train, epochs=10, batch_size=64, validation_data=(np.array(X_val.tolist()), y_val))

# Évaluation du modèle avec scikit-learn
predictions = model.predict(np.array(df_test['resized_image'].tolist()))
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)

accuracy = accuracy_score(true_labels, predicted_labels)
classification_report_str = classification_report(true_labels, predicted_labels)


print("Final Training Accuracy:", history.history['accuracy'][-1])
print("Final Training Loss:", history.history['loss'][-1])
print("Final Validation Accuracy:", history.history['val_accuracy'][-1])
print("Final Validation Loss:", history.history['val_loss'][-1])

print(f'Accuracy: {accuracy * 100:.2f}%')


In [ ]:
import matplotlib.pyplot as plt

# Visualiser quelques exemples du jeu de données
num_examples = 8  # Changer le nombre d'exemples à afficher

# Définir la disposition des sous-graphiques
rows = 2  # Changer le nombre de lignes en fonction du nombre d'exemples
cols = 4

# Créer la figure et les sous-graphiques
fig, axes = plt.subplots(rows, cols, figsize=(12, 6))

for i in range(num_examples):
    # Sélectionnez une image aléatoire du jeu de données
    sample_index = np.random.randint(len(df_train))
    sample_image = df_train['resized_image'].iloc[sample_index].reshape(28, 28)

    # Affichez l'image sur l'axe correspondant
    ax = axes.ravel()[i]
    ax.imshow(sample_image, cmap='gray')
    ax.set_title(f"Label: {df_train['label'].iloc[sample_index]}")

# Ajustez l'espacement entre les sous-graphiques
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# ...
class_names=[0,1,2,3,4,5,6,7,8,9]
# Fonction pour effectuer la prédiction sur une image spécifique
def predict_on_image(model, image_index, class_names):
    # Sélectionner une image du jeu de test (par exemple, la première image)
    input_image = df_test['resized_image'].iloc[image_index]
    true_label = y_test[image_index]
    
    # Afficher l'image
    plt.imshow(input_image.reshape(28, 28), cmap='gray')  # reshape pour s'assurer que les dimensions sont correctes
    plt.title(f'True Label: {class_names[np.argmax(true_label)]}')
    plt.show()
    
    # Prétraiter l'image
    input_image = np.expand_dims(input_image, axis=0)  # Ajouter la dimension du lot
    
    # Faire la prédiction
    prediction = model.predict(input_image)
    predicted_label = np.argmax(prediction)
    
    # Afficher le résultat de la prédiction
    print(f'La prédiction du modèle est : {class_names[predicted_label]}')

# Utiliser la fonction pour prédire sur une image spécifique (par exemple, image numéro 5)